# Specialize For SVJAIR

In [36]:
import pandas as pd
import time
import csv
import requests
import datetime
import json
import os

class GetSjvAir:
    """
        This Class is for SJV AIr
        initialize GetSjvAir --> inputs are self, list_sensor,start_date, end_date, data
        list_sensor --> list of sensor ID's / names
        start_date / end_date --> can just do date in this format: YYYY-MM-DD
        you can also add time by putting it next to the date in this format --> 00:00:00 (24 hour format)
        data --> list of data types you want (ex: timestamp, celsius, pressure, etc.)
    """
    def __init__(self , list_sensor , start_date , end_date , data): 
        self.list_sensor = list_sensor
        self.start_date = start_date
        self.end_date = end_date
        self.data = data        
                
    def get_data_json_sjvair(self , name):
        """
            This function is for getting sensor name , 
            lattitudes, and longitude.
            The api will be in json file
        """
        response = requests.get(f"https://www.sjvair.com/api/1.0/monitors/{name}")
            
        sensor_data = response.json()
        
        result_list = {}
        
        #if the user wants the name given to the sensor
        if "name" in self.data:

            result_list = {
                'sensor_id': name,
                'name': sensor_data['data']['name'],
            }
            
        #if the user wants the coordinates of the sensor
        elif "log" in self.data or "lat" in self.data:
            result_list = {
                'sensor_id': name,
                'log': sensor_data['data']['position']['coordinates'][0],
                'lat': sensor_data['data']['position']['coordinates'][1],
            }
        
        return result_list
        
    def get_data_sjvair(self):
        """
            SJVAir doens't give the option to choose what type of data 
            the user wants when getting the CSV data
            So this function gets the full CSV file and then
            makes a new one with only the data the user asked for
        """
        
        parameters = {'timestamp__gte': self.start_date, 'timestamp__lt': self.end_date}
        
        if not os.path.exists("svjair_folder"):
            os.makedirs("svjair_folder")

        for i in self.list_sensor:
            
            all_data = []
            
            csv_response = requests.get(f"https://www.sjvair.com/api/1.0/monitors/{i}/entries/csv/", params=parameters)
            
            with open(f"svjair_folder/SVJAir_{i}.csv", "wb") as f:
                f.write(csv_response.content)

            data_df = pd.read_csv(f"svjair_folder/SVJAir_{i}.csv")
            
            result_list = self.get_data_json_sjvair(name=i)
            
            for j in range(len(data_df)):
                
                val = data_df.iloc[j]      # set the val as series of the data_df
                
                val_data = {}      
                
                for k in self.data:
                    
                    try:
                        
                        val_data[k] = val[k]
                        
                    except KeyError:
                        
                        continue
                    
                val_data.update(result_list)
                
                all_data.append(val_data)

            df_final = pd.DataFrame(all_data)
            
            df_final.to_csv(f"svjair_folder/SVJAir_{i}.csv", index=False)
            
            print(f"Finish Download svjair_folder/SVJAir_{i}.csv")
            
############################################################################


# Specialize For PurpleAIr

In [37]:
class GetPurpleAirData:
    """
        This Class is for Purple Air
        initialize GetPurpleAirData --> inputs are sensor_list, start_date, end_date, headers, fields
        #sensor_list --> list of ID numbers for the sensors
        #start_date / end_date --> datetime.datetime(YYYY, MM, DD, 00,00,00)
        #you can also do it without adding the time
        #headers --> API key
    """
    def __init__(self, sensor_list, start_date, end_date, headers, fields):
        self.sensor_list = sensor_list
        self.start_date = start_date
        self.end_date = end_date
        self.headers = headers
        self.fields = fields
        
    def convert_date_time(self , timestampunix):
        """
            Convert Unix time into normal timestamp format
        """
        
        pst_timezone = datetime.timezone(datetime.timedelta(hours=-8))
        converted_time = datetime.datetime.fromtimestamp(timestampunix, tz= datetime.timezone.utc)
        converted_time_pst = converted_time.astimezone(pst_timezone)
        converted_time_pst.isoformat()
        return converted_time_pst
        
    def clean_date(self , file_name):
        """
            cleans the returned date from convert_date_time function 
            and puts it into the csv file
        """
        
        df = pd.read_csv(file_name)
                    
        df['time_stamp'] = df['time_stamp'].apply(self.convert_date_time)
            
        df = df.sort_values(by='time_stamp', ascending=True)
            
        df.to_csv(file_name, index=False)

    def get_data_purpleair(self):
        """
            This function use for getting data from the purple air api
        """
        for index in range(len(self.sensor_list)):

            url = f'https://api.purpleair.com/v1/sensors/{self.sensor_list[index]}/history/csv?fields={",".join(self.fields)}'
            
            try:
                
                response = requests.get(url, headers=self.headers)
                
                response.raise_for_status()
                
                #if this directory doesn't exist, then it will make it
                if not os.path.exists("purpleair_folder"):
                    os.makedirs("purpleair_folder")
                    
                filename = f"purpleair_folder/sensor_{self.sensor_list[index]}_data.csv"
                
                with open(filename , "wb") as f:
                    
                    f.write(response.content)
                    
                print(f"Finish Download {filename}")
                
                self.clean_date(filename)
                
            #prints out errors from the api such as out of tokens
            except requests.HTTPError as http_err:
                print(http_err)
                
            except Exception as err:
                
                print(err)


# Main Program

In [39]:

if __name__ == '__main__':
    
    datatypes = input("(SJVAIR OR PurpleAir)")
    
    if datatypes.upper() == 'SJVAIR':
        
        data = GetSjvAir(
            list_sensor=["yiz5lVBeRf26NdX-KtrPlg" , "K6rfX7COTLKuQIFIh00gjA"] , 
            start_date = "2023-11-13" , 
            end_date= "2023-11-15" , 
            data = ['timestamp' , "celsius" , "pm25" , "humidity" , "name" , "pressure"] , 
        )
        
        data.get_data_sjvair()
        
    elif datatypes.upper() == 'PURPLEAIR':
        
        data = GetPurpleAirData(
            sensor_list=["155683", "104106"],
            start_date=datetime.datetime(2023, 11, 16,4,30,0),
            end_date=datetime.datetime(2023, 11, 16,7,0,0),
            headers={'X-API-Key': ""},
            fields=["pm2.5_alt","pm2.5_atm","pressure","humidity"]
        )
        
        data.get_data_purpleair()
        
    else:
        
        print("Please Choose Between PurpleAir or SvjAir")

Finish Download purpleair_folder/sensor_155683_data.csv
Finish Download purpleair_folder/sensor_104106_data.csv
